# Bob's Discount Furniture

Warning: because some data products are inconsistent (changing JSON formats, etc...), not all code cleanly refactorable.

In [ ]:
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from ExtractedSpectrum import ExtractedSpectrum
from nice_tables import LatexTable
import xspec_utils as xs

%cd results_spec

In [ ]:
def val_pnerrs(model_dict, comp, par):
    """Convenience method to extract
    value, +ve error, -ve error
    from a model dict as returned by xspec_utils.model_dict.
    Errors are signed (i.e., -ve error value is < 0)
    
    Arguments
        model_dict: xspec_utils fit dict (i.e., JSON dict)
        comp: component name (string)
    Returns:
        value, +ve error, -ve error three-tuple
    """
    p = model_dict[comp][par]
    val = p['value']
    pos_err = p['error'][1] - p['value']
    neg_err = p['error'][0] - p['value']
    return val, pos_err, neg_err

## Estimate masses from emission measures

In [ ]:
def volfrac(r1,r2):
    """Volume fraction of a sphere subtended by an annulus at scaled radii r1, r2.
    Requires r1, r2 both in range [0,1] and r1 <= r2.
    Will not give sensible answers otherwise.
    
    This is the result of integral
    \int_{0}^{2\pi} d\phi \int_{r_1}^{r_2} r dr \int_{-\sqrt{R^2-r^2}}^{\sqrt{R^2-r^2}} dz
    """
    return (1 - r1**2)**(3/2) - (1 - r2**2)**(3/2)

def pixel2sqarcsec(pixels):
    """Convert from XMM detector pixels to sq. arcsec"""
    return pixels * 0.05**2

def annulus_area(r1,r2):
    """Yeah"""
    return np.pi * (r2**2 - r1**2)

print "  0-100\": {:.3f}".format(volfrac(0, 0.25))
print "100-200\": {:.3f}".format(volfrac(0.25, 0.50))
print "200-300\": {:.3f}".format(volfrac(0.50, 0.75))
print "300-400\": {:.3f}".format(volfrac(0.75, 1))

In [ ]:
THETA_G309 = 6 * (1/60) * np.pi/180  # 6 arcminutes -> radians
M_SUN = 1.988e33  # grams (source: wolfram|alpha)
M_H = 1.674e-24  # grams (source: wolfram|alpha)
D_5KPC = 1.543e22  # 5 kiloparsecs in cm (source: Wolfram|alpha)

def density_scale(norm, angular_radius, f_subtend):
    """Solves for density from XSPEC apec norm (see http://atomdb.org/faq.php)

    Assume a spherical, relatively compact source in the sky.
    Let $\eta_0$ be the corrected XSPEC norm (corrected for chip gaps etc)
    that describes emission for a fraction f_subtend of this sphere.
    Then:

        n_h ~ \sqrt{ \frac{ 10^{14} \eta_0 }{ 0.4 \theta^3 (5 \unit{kpc}) } } f^{-1/2} d_{5}^{-1/2}

    Arguments:
        norm (cm^-5): XSPEC emission measure 10^{-14} * \frac{1}{4\pi D^2} \int n_e n_H dV
        angular_radius (radians): radius of entire source
        f_subtend: fraction of spherical source subtended (range [0,1])
    Output:
        density estimate, scaled to filling factor 1 and distance 5 kpc
        I.e., n_h ~ OUTPUT * f^{-1/2} d_{5}^{-1/2}
    """

    return np.sqrt( 1e14 * norm / (0.4 * angular_radius**3 * D_5KPC) )

def nh2mass(n_h_scale, volume):
    """Solve for emitting mass (again, scaled to 5kpc and f=1)
    using expression from Auchettl+ 2015
    
        M = 1.4 n_H m_H f V
    
    (not sure where factor 1.4x comes from, guessing metallicity)
    Arguments:
        n_h_scale (cm^{-3}): density scaled to 5kpc, f=1 from density_scale(...)
        volume (cm^3): source volume under consideration _assuming d=5kpc_
    Output:
        mass estimate, scaled to filling factor 1 and distance 5 kpc
        I.e., M ~ OUTPUT * f^{1/2} d_{5}^{+5/2}
    """
    return 1.4 * n_h_scale * M_H * volume

In [ ]:
# Please note: these constant factors are for 0087940201 MOS1 only
# But, the process of "upscaling" yields norms that are basically
# scaled to that expected for the perfect annulus, without pt source removals, chip gaps, etc
# and are INDEPENDENT of the exposure. good.
for ann in [(0,100), (100,200), (200,300), (300,400)]:
    reg = "ann_{:03d}_{:03d}".format(*ann)
    fd = xs.load_dict('20160708_fourann_center-mg-free_snr_' + reg + '.json')
    md = fd['comps']
    
    extr = ExtractedSpectrum("0087940201", "mos1S001", reg)
    print reg
    print '  backscal / annulus_area = {:.1f} / {:.1f} = {:.5f}'.format(
            pixel2sqarcsec(extr.backscal()), annulus_area(*ann),
            pixel2sqarcsec(extr.backscal()) / annulus_area(*ann))
    
    norm = md['vnei']['norm']['value']
    norm_corr = (norm * md['constant']['factor']['value']
                 * annulus_area(*ann) / pixel2sqarcsec(extr.backscal()))
    print '  XSPEC norm:', norm
    print '  norm * constant * (annulus_area / backscal):', norm_corr
    
    nH_scaled = density_scale(norm_corr, THETA_G309, volfrac(ann[0]/400, ann[1]/400))
    mass = nh2mass(nH_scaled, volfrac(ann[0]/400, ann[1]/400) * 4/3*np.pi*(THETA_G309 * D_5KPC)**3)
    print '  Density scale for f=1, D=5kpc:', density_scale(norm_corr, THETA_G309, volfrac(ann[0]/400, ann[1]/400))
    print '  Inferred mass scale (units: Msun):', mass / M_SUN
    print

In [ ]:
fd = xs.load_dict('20160713_src_bkg_mg.json')
extr = ExtractedSpectrum("0087940201", "mos1S001", 'src')
print 'Integrated source'
print '  backscal / annulus_area = {:.1f} / {:.1f} = {:.5f}'.format(
        pixel2sqarcsec(extr.backscal()), annulus_area(0, 400),
        pixel2sqarcsec(extr.backscal()) / annulus_area(0, 400))

norm = fd['1']['snr_src']['vnei']['norm']['value']
norm_bounds = fd['1']['snr_src']['vnei']['norm']['error'][0:2]

norm_corr = (norm * fd['1']['snr_src']['constant']['factor']['value']
             * annulus_area(0, 400) / pixel2sqarcsec(extr.backscal()))

nH_scaled = density_scale(norm_corr, THETA_G309, volfrac(ann[0]/400, ann[1]/400))
mass = nh2mass(nH_scaled, 4/3*np.pi*(THETA_G309 * D_5KPC)**3)

print '  XSPEC norm: {:.5f} (range: [{:5f}, {:5f}])'.format(norm, *norm_bounds)
print '  norm * constant * (annulus_area / backscal):', norm_corr
print '  Density scale for f=1, D=5kpc:', nH_scaled
print '  Inferred mass scale (units: Msun):', mass / M_SUN

mg_abund = fd['1']['snr_src']['vnei']['Mg']['value']
si_abund = fd['1']['snr_src']['vnei']['Si']['value']
s_abund =  fd['1']['snr_src']['vnei']['S']['value']

# Abundances from Table 2 of Wilms+ (2000), using estimated ISM abundances
print
print '  Inferred Mg mass scale, ISM+ejecta (units: Msun): ', (mg_abund / 1.4) * 10**(7.40 - 12) * 24 * mass / M_SUN
print '  Inferred Si mass scale, ISM+ejecta (units: Msun): ', (si_abund / 1.4) * 10**(7.27 - 12) * 28 * mass / M_SUN
print '  Inferred S  mass scale, ISM+ejecta (units: Msun): ', (s_abund / 1.4) * 10**(7.09 - 12) * 32 * mass / M_SUN

## Soft proton parameters as a function of radius

In [ ]:
fd = xs.load_dict("20160701_fiveann.json")

In [ ]:
latex_hdr = [['Annulus', '2001 MOS', '2001 PN', '2009 MOS']]
latex_cols = ['{:s}', 0, 0, 0]
ltab = LatexTable(latex_hdr, latex_cols, "Soft proton power law indices", prec=2)

labels = ['0-100', '100-200', '200-300', '300-400', '400-500']
spectra = [np.array([1,3,5]),
           np.array([1,3,5]) + 5,
           np.array([1,3,5]) + 10,
           np.array([1,3,5]) + 15,
           np.array([1,3,5]) + 20]
spectra = [map(str, x) for x in spectra]

for lab, indices in zip(labels, spectra):
    ltr = [lab]
    for idx in indices:
        ltr.append(fd[idx]['sp']['powerlaw']['PhoIndex']['value'])
    ltab.add_row(*ltr)

print ltab

Hard to see for 0087940201 MOS1 case, but 200-300 powerlaw basically overlaps 400-500.
Behavior is broadly consistent with what we expect, actually, which is very reassuring.  Fits also get jumpier going outwards in radius.  But, outer annuli encompass larger annuli -- so the counts should still be pretty comparable.  I would actually expect outer annuli fits to be a bit better constrained

In [ ]:
x = np.logspace(-1, 1)

for offset, exposure in enumerate(['0087940201 MOS1', '0087940201 MOS2', '0087940201 PN',
                                   '0551000201 MOS1','0551000201 MOS2']):
    print exposure
    for idx, lab in zip(np.array([1, 6, 11, 16, 21]) + offset,
                        ['0-100', '100-200', '200-300', '300-400', '400-500']):
        idx = str(idx)
        phoindex = fd[idx]['sp']['powerlaw']['PhoIndex']['value']
        norm = fd[idx]['sp']['powerlaw']['norm']['value']
        print "{:s}: index {:.2f}, norm {:.2e}".format(lab, phoindex, norm)
        plt.loglog(x, norm * x**(-1 * phoindex), label=lab)

    plt.legend(loc='best')
    plt.xlabel("Energy (keV)")
    plt.ylabel(r'Effective photons s$^{-1}$ cm$^{-2}$ keV$^{-1}$')
    plt.show()

## vnei fit parameters as a function of radius

In [ ]:
fd = xs.load_dict('20160708_fourann_center-mg-free_snr_ann_000_100.json')
fd['comps'].keys()

In [ ]:
fd = xs.load_dict("20160701_fiveann.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([50, 150, 250, 350], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='5')

vals = []
p_errs = []
n_errs = []
# Old style json dumps
for fname in ['20160708_fourann_center-mg-free_snr_ann_000_100.json',
              '20160708_fourann_center-mg-free_snr_ann_100_200.json',
              '20160708_fourann_center-mg-free_snr_ann_200_300.json',
              '20160708_fourann_center-mg-free_snr_ann_300_400.json']:
    fd = xs.load_dict(fname)
    c = fd['comps']['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
print vals, p_errs, n_errs
plt.errorbar([48, 148, 248, 348], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg')

fd = xs.load_dict("20160708_fourann_center-mg-o-free.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['kT']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([52, 152, 252, 352], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg,O')

plt.legend(loc='best')
plt.xlim(-5,405)
plt.xlabel("Radius (arcseconds)")
plt.ylabel(r'Plasma electron temperature (keV)')
plt.savefig('fig_kt_radius.pdf')
plt.show()

Copy paste code for ionization timescale Tau

In [ ]:
fd = xs.load_dict("20160701_fiveann.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['Tau']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([50, 150, 250, 350], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='5')

vals = []
p_errs = []
n_errs = []
# Old style json dumps
for fname in ['20160708_fourann_center-mg-free_snr_ann_000_100.json',
              '20160708_fourann_center-mg-free_snr_ann_100_200.json',
              '20160708_fourann_center-mg-free_snr_ann_200_300.json',
              '20160708_fourann_center-mg-free_snr_ann_300_400.json']:
    fd = xs.load_dict(fname)
    c = fd['comps']['vnei']['Tau']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
print vals, p_errs, n_errs
plt.errorbar([48, 148, 248, 348], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg')

fd = xs.load_dict("20160708_fourann_center-mg-o-free.json")
vals = []
p_errs = []
n_errs = []
for idx, model in zip(np.array([1, 6, 11, 16]),
                  ['snr_ann_000_100', 'snr_ann_100_200', 'snr_ann_200_300', 'snr_ann_300_400']):
    c = fd[str(idx)][model]['vnei']['Tau']
    vals.append(c['value'])
    p_errs.append(c['error'][1] - c['value'])
    n_errs.append(abs(c['error'][0] - c['value']))
plt.errorbar([52, 152, 252, 352], vals, yerr=[n_errs, p_errs], xerr=50, marker='o', ls='', label='4Mg,O')

plt.legend(loc='best')
plt.xlim(-5,405)
plt.xlabel("Radius (arcseconds)")
plt.ylabel(r'Ionization timescale ($\mathrm{s\;cm^{-3}}$)')
plt.savefig('fig_tau_radius.pdf')
plt.show()

## Source fit parameters

Todo: just regenerate all the joint source-background fits with new products.
Saves fair bit of trouble, should be straightforward.

Please note: final table in .tex file has been significantly modified; presentation requires accompanying comment to explain units (had to strip to prevent table from overflowing page width...)

In [ ]:
fd = xs.load_dict("20160713_src_bkg_mg.json")
md = fd['1']['snr_src']
md_xrb = fd['1']['xrb']

In [ ]:
latex_hdr = [[r'$N_\mt{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$kT$', '(keV)'],
             [r'$\tau$', r'($10^{10} \unit{s\;cm^{-3}}$)'],
             ['Mg', '(-)'],
             ['Si', '(-)'],
             ['S', '(-)'],
             ['EM (scaled)', r'($10^{-14} \unit{cm^{-5}}$)'],
             [r'$k_B T_{\mt{local}}$', '(keV)'],  # XRB parameters
             [r'$N_\mt{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$k_B T_{\mt{halo}}$', '(keV)'],
             [r'$\chi^2_{\mt{red}}$', ''],
             [r'$\chi^2 / (\mt{dof})$', '']]
latex_hdr = np.array(latex_hdr).T

latex_cols = [2, 2, 2, 2, 2, 2, 2] + [2, 2, 2] + ['{:.3f}', '{:s}']
ltab = LatexTable(latex_hdr, latex_cols, "Integrated source with Mg free", prec=2)

ltr = []
ltr.extend(val_pnerrs(md, 'tbnew_gas', 'nH'))
ltr.extend(val_pnerrs(md, 'vnei', 'kT'))
ltr.extend(np.array(val_pnerrs(md, 'vnei', 'Tau')) / 1e10)
ltr.extend(val_pnerrs(md, 'vnei', 'Mg'))
ltr.extend(val_pnerrs(md, 'vnei', 'Si'))
ltr.extend(val_pnerrs(md, 'vnei', 'S'))
ltr.extend(val_pnerrs(md, 'vnei', 'norm'))

ltr.extend(val_pnerrs(md_xrb, 'apec', 'kT'))
ltr.extend(val_pnerrs(md_xrb, 'tbnew_gas', 'nH'))
ltr.extend(val_pnerrs(md_xrb, 'apec_5', 'kT'))

ltr.append(fd['fitStat'] / fd['dof'])
ltr.append("{:.2f} / {:d}".format(fd['fitStat'], fd['dof']))

ltab.add_row(*ltr)

print ltab
print val_pnerrs(md, 'vnei', 'norm')  # Didn't format correctly

In [ ]:
% cat "20160701_src_bkg.tex"

## Four annuli with varied center abundances

In [ ]:
fdictfs = ["20160708_fourann_center-mg-free_snr_ann_000_100.json",
          "20160708_fourann_center-mg-o-free_snr_ann_000_100.json",
          "20160708_fourann_center-mg-ne-free_snr_ann_000_100.json",
          "20160708_fourann_center-mg-o-ne-free_snr_ann_000_100.json",
          "20160708_fourann_center-mg-fe-free_snr_ann_000_100.json",
          "20160712_fourann_center-mg-o-fe-free.json"
          ]
# fdictf = fit dict file
labels = ["4Mg", "4Mg,O", "4Mg,Ne", "4Mg,Ne,O", "4Mg,Fe", "4Mg,O,Fe"]

In [ ]:
def val_pnerrs(model_dict, comp, par):
    """Convenience method to extract
    value, +ve error, -ve error
    from a model dict as returned by xspec_utils.model_dict.
    Errors are signed (i.e., -ve error value is < 0)
    
    Arguments
        model_dict: xspec_utils fit dict (i.e., JSON dict)
        comp: component name (string)
    Returns:
        value, +ve error, -ve error three-tuple
    """
    p = model_dict[comp][par]
    val = p['value']
    pos_err = p['error'][1] - p['value']
    neg_err = p['error'][0] - p['value']
    return val, pos_err, neg_err

In [ ]:
latex_hdr = [['Annulus', ''],
             [r'$n_\mathrm{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$kT$', r'(keV)'],
             [r'$\tau$', r'($10^{10} \unit{s\;cm^{-3}}$)'],
             ['O', '(-)'],
             ['Ne', '(-)'],
             ['Mg', '(-)'],
             ['Si', '(-)'],
             ['S', '(-)'],
             ['Fe', '(-)'],
             [r'$\chi^2_{\mt{red}}$', r'$\chi^2 / (\mt{dof})$']]
latex_hdr = np.array(latex_hdr).T

latex_cols = ['{:s}', 2, 2, 2, 2, 2] + 4 * [2] + ['{:s}']  # O, Ne, Mg, Fe; chisqred
ltab = LatexTable(latex_hdr, latex_cols, "G309.2-0.6 annuli fit with errors", prec=2)

for fdictf, lab in zip(fdictfs, labels):
    
    fd = xs.load_dict(fdictf)
    old = "20160708" in fdictf
    if old:
        md = fd['comps']  # Note that old naming convention confused 'comps' with xspec models
    else:
        md = fd["1"]["snr_ann_000_100"]
    
    ltr = [lab]
    ltr.extend(val_pnerrs(md, 'tbnew_gas', 'nH'))
    ltr.extend(val_pnerrs(md, 'vnei', 'kT'))
    ltr.extend(np.array(val_pnerrs(md, 'vnei', 'Tau')) / 1e10)
    ltr.extend(val_pnerrs(md, 'vnei', 'O'))
    ltr.extend(val_pnerrs(md, 'vnei', 'Ne'))
    ltr.extend(val_pnerrs(md, 'vnei', 'Mg'))
    ltr.extend(val_pnerrs(md, 'vnei', 'Si'))
    ltr.extend(val_pnerrs(md, 'vnei', 'S'))
    ltr.extend(val_pnerrs(md, 'vnei', 'Fe'))
    
    if old:
        ltr.append("{:.3f} = {:.2f} / {:d}".format(fd['fitstat'][1] / fd['dof'], fd['fitstat'][1], fd['dof']))
    else:
        ltr.append("{:.3f} = {:.2f} / {:d}".format(fd['fitStat'] / fd['dof'], fd['fitStat'], fd['dof']))

    ltab.add_row(*ltr)

print ltab.__str__().replace('${1.00}^{-1.00}_{-1.00}$', '    ')

## Tabulate 4 & 5 annuli fit parameters

Currently, no "four annulus stock" fit JSON available; use pre-generated table

In [ ]:
# Nice LaTeX table

fd = xs.load_dict("20160701_fiveann.json")
rings = []
for idx, model in zip([1, 6, 11, 16, 21], ['snr_ann_000_100', 'snr_ann_100_200',
                                           'snr_ann_200_300', 'snr_ann_300_400', 'snr_ann_400_500']):
    rings.append(fd[str(idx)][model])

def ring_val_errs(ring, cname, pname):
    p = ring[cname][pname]
    val = p['value']
    pos_err = p['error'][1] - p['value']
    neg_err = p['error'][0] - p['value']
    return val, pos_err, neg_err

latex_hdr = [['Annulus', ''],
             [r'$n_\mathrm{H}$', r'($10^{22} \unit{cm^{-2}}$)'],
             [r'$kT$', r'(keV)'],
             [r'$\tau$', r'($10^{10} \unit{s\;cm^{-3}}$)'],
             ['Si', '(-)'],
             ['S', '(-)']]
latex_hdr = np.array(latex_hdr).T

latex_cols = ['{:s}', 2, 2, 2, 2, 2]
ltab = LatexTable(latex_hdr, latex_cols, "Five annulus fit", prec=2)

for ring in rings:
    ltr = [ring['name']]
    ltr.extend(ring_val_errs(ring, 'tbnew_gas', 'nH'))
    ltr.extend(ring_val_errs(ring, 'vnei', 'kT'))
    ltr.extend(np.array(ring_val_errs(ring, 'vnei', 'Tau')) / 1e10)
    ltr.extend(ring_val_errs(ring, 'vnei', 'Si'))
    ltr.extend(ring_val_errs(ring, 'vnei', 'S'))

    ltab.add_row(*ltr)

print ltab

This will be regenerated, with correct emission measures shortly.

In [ ]:
%cat "20160706_fourann_stock.tex"